This notebook loads cluster FRAP data from the output of the "Extract_two_radii_TrackMate.ijm" ImageJ macro, then plots and fits the FRAP data.

In [ ]:
# load the built-in and custom modules

# uncomment for debugging
%load_ext autoreload 
%autoreload 2 

import os, sys, inspect
import matplotlib.pylab as plt
import numpy as np
from pprint import pprint
import glob
import pandas as pd


# Add source code directory (src) to path to enable module import
curr_frame = inspect.getfile(inspect.currentframe())
curr_dir = os.path.dirname(os.path.abspath(curr_frame))
parent_dir = os.path.dirname(curr_dir)
module_dir = os.path.join(parent_dir, 'src')
os.sys.path.insert(0, module_dir)

# import fraptools
import fraptools as frap


In [ ]:
# Load data

# Path to FRAP data (text files saved by "Extract_two_radii_TrackMate.ijm")
data_dir = '../data/processed/IRE1_foci_FRAP_2018-10-11/spot_radii'

frap_files = sorted(glob.glob(os.path.join(data_dir,'*.txt')))

# list of all FRAP data:
frap_data_by_file = []
filenames_no_ext = []

# Go file by file and read data
for file in frap_files:
    
    # Read data from the provided source file
    data = pd.read_csv(file, delimiter='\t')
    filename_no_ext = os.path.split(os.path.splitext(file)[0])[1]
    frap_data_by_file.append(data)
    filenames_no_ext.append(filename_no_ext)


In [ ]:
# Analyze and plot the foci FRAP data
# Plotting and figure saving params
plot_figs = True
save_figs = True
save_dir = '../reports/figures/IRE1_foci_FRAP_2018-10-11'

# The excluded trace file allows you to manually remove bad traces from the analysis.
# It should be a simple csv with each line containing the file name followed by trace ID
excluded_trace_file = '../data/processed/IRE1_foci_FRAP_2018-10-11/2018-10-11_excluded_traces.csv'

# Input imaging parameters:
frame_interval = 5 # in seconds #0.2
bleach_n_frames = 8 # how many frames bleaching takes (max)


# interactive plotting
if plot_figs: 
    %matplotlib    
    

# Populate excluded trace file
excluded_files_df = pd.read_csv(excluded_trace_file)
excluded_traces = [tuple(x) for x in excluded_files_df.values]

# create save figure dirs
if save_figs:
    save_dir_png = os.path.join(save_dir, 'png')
    save_dir_svg = os.path.join(save_dir, 'svg')
    if not os.path.exists(save_dir_png):
        os.makedirs(save_dir_png)
    if not os.path.exists(save_dir_svg):
        os.makedirs(save_dir_svg)

        
# Extract individual traces from the raw data
df_by_trace, corr_ints, trace_IDs = frap.get_traces_from_df_list(frap_data_by_file, 
                                    filenames_no_ext, exclude=excluded_traces)

# Fit the individual FRAP traces
fit, data = frap.fit_frap_smart(corr_ints, frame_interval, bleach_n_frames)

    
# Plot results

for f,d,trace_ID in zip(fit, data, trace_IDs):
    
    file_name = trace_ID[0]
    trace_num = trace_ID[1]
    
    full_name = file_name + '_trace-ID_' + str(trace_num)
    
    fig, axarr = frap.plot_fit_results(f,d)
    fig.canvas.set_window_title(full_name)
    plt.suptitle(full_name)
    if save_figs:
        fig_filename_svg = os.path.join(save_dir_svg, (full_name+'.svg'))
        fig_filename_png = os.path.join(save_dir_png, (full_name+'.png'))
        plt.savefig(fig_filename_svg)
        plt.savefig(fig_filename_png)


In [ ]:
# Summarize fit results
save_dir = '../reports/figures/IRE1_foci_FRAP_2018-10-11/summary'
# create save figure dirs
if save_figs:
    save_dir_png = os.path.join(save_dir, 'png')
    save_dir_svg = os.path.join(save_dir, 'svg')
    if not os.path.exists(save_dir_png):
        os.makedirs(save_dir_png)
    if not os.path.exists(save_dir_svg):
        os.makedirs(save_dir_svg)

%matplotlib

frap_fits_foci = fit

thalf_foci = [f['thalf'] for f in frap_fits_foci]

mobile_f_foci = [f['mobile_fraction'] for f in frap_fits_foci]   
#thalf_er = [f['thalf'] for f in frap_fits_er]
#mobile_f_er = [f['mobile_fraction'] for f in frap_fits_er]   

#print(np.mean(thalf_foci), np.mean(mobile_f_foci))
#print(np.mean(thalf_er), np.mean(mobile_f_er))

labels = ('Foci')

plt.rcParams["figure.figsize"] = [20,8]
f2, axarr = plt.subplots(1,3, sharey=False)
axarr[0].set_title('Half-time of recovery')
axarr[0].set_ylabel('Time (s)')
axarr[1].set_ylabel('Mobile fraction')
axarr[1].set_title('IRE1 Mobile fraction')
axarr[0].boxplot(thalf_foci)
axarr[1].boxplot(mobile_f_foci)

if save_figs:
    fig_filename_svg = os.path.join(save_dir_svg, 'Foci_IRE1_recovery.svg')
    fig_filename_png = os.path.join(save_dir_png, 'Foci_IRE1_recovery.png')
    plt.savefig(fig_filename_svg)
    plt.savefig(fig_filename_png)

plt.show()